In [1]:
import xarray as xr
import scipy.io as sio
from scipy.fft import fft, ifft
from matplotlib.path import Path
import pickle
from scipy.stats import pearsonr
from os.path import join
import numpy as np
from glob import glob
import matplotlib as mpl
import matplotlib.pyplot as plt
import cmocean
import cartopy.feature as cfeature
import matplotlib.patches as mpatches
import cartopy.crs as ccrs
import pandas as pd
from copy import deepcopy
    
### Specify figure resolution
scale = 1
mpl.rcParams['figure.dpi']        = 150 # set figure resolution (dots per inch) 
mpl.rcParams['hatch.linewidth']   = .15  * scale
mpl.rcParams['axes.labelsize']    = 11  * scale
mpl.rcParams['axes.titlesize']    = 13  * scale
mpl.rcParams['xtick.labelsize']   = 9   * scale
mpl.rcParams['ytick.labelsize']   = 9   * scale
mpl.rcParams['lines.linewidth']   = 1   * scale
mpl.rcParams['legend.fontsize']   = 8   * scale
mpl.rcParams['patch.linewidth']   = 1   * scale
mpl.rcParams['contour.linewidth'] =.5   * scale
mpl.rcParams['axes.labelpad']     = 4 * scale   # space between label and axis
mpl.rcParams['xtick.major.pad']   = 0 * scale   # distance to major tick label in points
mpl.rcParams['ytick.major.pad']   = mpl.rcParams['xtick.major.pad']
mpl.rcParams['lines.markersize']  = 3 * scale

### cartopy-specific parameters
gridline_width                    = .7 * scale
border_width                      = .3 * scale

### manually specified colorbar params
tick_width                        = 0 * scale # controls size of tickmark next to labels
tick_length                       = 2 * scale

$\zeta = \frac{1}{R\cos{\theta}}\Big[\frac{\partial}{\theta}\left(u\cos{\theta}\right) - \frac{\partial v}{\partial \phi}\Big]$

$\zeta = \frac{1}{R\cos{\theta}} \Big[\frac{\partial}{\partial\theta}\left(u\cos{\theta}\right)- \frac{\partial v}{\partial \phi}\Big]$

convert coords to polar (i.e. deg 2 rad)

In [77]:
u = xr.open_dataset('/Users/theo/Downloads/era5_u_843.nc').u.sel(latitude=slice(50,0),longitude=slice(220,260)).compute()
v = xr.open_dataset('/Users/theo/Downloads/era5_v_843.nc').v.sel(latitude=slice(50,0),longitude=slice(220,260)).compute()

lon = deepcopy(u.longitude)
lat = deepcopy(u.latitude)

theta = np.deg2rad(u.latitude)
phi   = np.deg2rad(u.longitude)

u['longitude'] = phi
v['longitude'] = phi

u['latitude']  = theta
v['latitude']  = theta

u0 = deepcopy(u)
v0 = deepcopy(v)

R = 6.37e6
dv_dphi             = -v0.differentiate('longitude')
u_cos_theta         = u0 * np.cos(u0.latitude)

dtheta_du_cos_theta = -u_cos_theta.differentiate('latitude')
zeta                = 1 / (R*np.cos(u0.latitude)) * (dtheta_du_cos_theta - dv_dphi)
zeta = zeta.compute()

In [93]:
idx = np.argsort(zeta.isel(level=1,time=0).stack(n=['latitude','longitude']).values)
zeta.isel(level=1,time=0).stack(n=['latitude','longitude']).values[idx[-5:]]

array([0.00051895, 0.00052834, 0.0005321 , 0.00053226, 0.00054353],
      dtype=float32)

In [96]:
zeta.isel(level=1,time=0).stack(n=['latitude','longitude'])[idx[-5:]]

<xarray.DataArray (n: 5)>
array([0.00051895, 0.00052834, 0.0005321 , 0.00053226, 0.00054353],
      dtype=float32)
Coordinates:
    level      int32 850
    time       datetime64[ns] 2017-07-20
  * n          (n) MultiIndex
  - latitude   (n) float64 0.3098 0.3142 0.3054 0.3098 0.3098
  - longitude  (n) float64 3.888 3.892 3.892 3.896 3.892

In [80]:
for t in zeta.time:
    x,y = np.argmax

<xarray.DataArray (latitude: 201, time: 48, level: 2, longitude: 161)>
array([[[[ 3.43321699e-05,  4.57587776e-05,  7.66716621e-05, ...,
           8.88659924e-05,  8.58726053e-05,  8.40961511e-05],
         [-4.40467920e-05, -3.64814478e-05, -2.10377275e-05, ...,
          -4.82075593e-05, -3.88300796e-05, -2.60223715e-05]],

        [[ 1.77341484e-04,  1.54462512e-04,  1.08277440e-04, ...,
           1.53576293e-05,  3.01155833e-05,  4.38626048e-05],
         [-5.38186623e-06, -5.70384557e-08,  6.68624716e-06, ...,
          -4.95598715e-06,  2.16339613e-05,  4.49815161e-05]],

        [[-5.00877213e-05, -5.49486176e-05, -6.17129626e-05, ...,
           2.69918164e-05,  2.33045594e-05,  1.91888685e-05],
         [ 8.13990700e-05,  7.18070805e-05,  5.46358751e-05, ...,
          -8.19892557e-06, -6.85305577e-06,  1.14633485e-06]],

        ...,

        [[-7.33871348e-05, -7.84015720e-05, -8.22135335e-05, ...,
          -2.33534884e-05, -1.85370518e-05, -1.67770031e-05],
         [-1.14131099e-05, -1.30802819e-05, -1.31056431e-05, ...,
...
         [-5.94185485e-06,  2.62501726e-06,  1.32712421e-05, ...,
          -1.56344267e-05, -1.65893853e-05, -1.80434563e-05]],

        ...,

        [[-8.74118723e-06, -5.16985574e-06,  2.97298720e-06, ...,
           3.35168602e-06,  3.89130628e-06,  4.40331860e-06],
         [-2.24791552e-06, -9.93128469e-06, -2.57558549e-05, ...,
           2.01379498e-06,  6.75862702e-06,  1.12490961e-05]],

        [[ 1.15539960e-05,  4.84735483e-06, -7.72137901e-06, ...,
           4.23142592e-06,  2.74615286e-06,  1.38916232e-06],
         [-9.52677874e-06, -2.25549343e-06,  2.02933643e-06, ...,
           1.33692820e-05,  1.90735136e-05,  2.54614679e-05]],

        [[ 5.28020973e-06,  4.70679424e-06,  4.23709753e-06, ...,
          -6.52265442e-07, -1.60818297e-06, -1.52760708e-06],
         [ 2.20916445e-05,  2.14210395e-05,  2.08565543e-05, ...,
           5.33736625e-07,  6.30337854e-06,  1.11503723e-05]]]],
      dtype=float32)
Coordinates:
  * latitude   (latitude) float32 0.8727 0.8683 0.8639 ... 0.008727 0.004363 0.0
  * longitude  (longitude) float32 3.84 3.844 3.848 3.853 ... 4.529 4.533 4.538
  * level      (level) int32 250 850
  * time       (time) datetime64[ns] 2017-07-20 ... 2017-07-31T18:00:00

In [97]:
t=40
for t in range(len(u.time)):
    figsize=(5,4)
    fig = plt.figure(figsize=figsize)
    ax = fig.add_subplot(projection=ccrs.PlateCarree(central_longitude=180))
    ax.set_extent([220,260,10,30], crs=ccrs.PlateCarree())
    ax.coastlines(linewidth=border_width)
    ax.add_feature(cfeature.BORDERS, zorder=1, linewidth = border_width)
    gl = ax.gridlines(draw_labels=True, linestyle='--', alpha=.1, linewidth=gridline_width, color='k', zorder=1.05)
    gl.top_labels = False
    gl.right_labels = False
    gl.xlabel_style = {'size': mpl.rcParams['xtick.labelsize']}
    gl.ylabel_style = {'size': mpl.rcParams['ytick.labelsize']}

    cp1 = ax.contourf(lon,lat, zeta.isel(time=t, level=1),                  
                      extend='both',  cmap='cmo.amp', levels=np.arange(0,10e-4,10e-5),transform=ccrs.PlateCarree())
    cb = fig.colorbar(cp1)

    #### Quiver plot
    n=7
    x,y = np.meshgrid(lon[::n].values, lat[::n].values)
    qv = ax.quiver(x,y, u.isel(time=t, level=1).values[::n,::n], v.isel(time=t, level=1).values[::n,::n], scale=.5e3,
                   color='black', alpha=.6, transform=ccrs.PlateCarree(), pivot='middle')
    fig.savefig(f'lab-figs/zoomed_{t:03d}.png')
    plt.close(fig)